# Plantnet Classification

*Andrieu Grégoire & Gille Cyprien*

In [ ]:
import torch
import torch.nn as nn
import os
import pandas as pd
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torchvision.transforms.functional as TF
from PIL import Image
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler, random_split
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tqdm.notebook import tqdm
import copy
import matplotlib.pyplot as plt

## Data

In [ ]:
train_dir_path = "../input/polytech-nice-data-science-course-2021/polytech/train"

### EDA

In [ ]:
# Put here EDA that shows that the classes are unbalanced, and data augmentation to fix this problem 
# cf http://pytorch.org/vision/main/transforms.html#automatic-augmentation-transforms

In [ ]:
# All the classes
classes = os.listdir(train_dir_path)

# compute the number of images per class
nums = {}
for plant_num in classes:
    nums[str(plant_num)] = len(os.listdir(train_dir_path + '/' + plant_num))

img_per_class = pd.DataFrame(nums.values(), index=nums.keys(), columns=["number of images"])

In [ ]:
index = [i for i in range(153)]
plt.figure(figsize=(15, 6))
plt.bar(index, [n for n in nums.values()], width=0.4)
plt.xlabel('Classes (random order)')
plt.ylabel('Quantity of images available')
plt.show()

In [ ]:
img_per_class.describe()

### Oversampling et Data Augmentation

In [ ]:
# when getting from the under-represented classes, transform (to avoid always giving the same images maybe)

class SmartAugmentationDataset(ImageFolder):
    def __init__(self, path, transform=None, aug_class_id=None):
        super().__init__(path, transforms.Compose([transforms.Resize([183, 183]), transforms.ToTensor()]))
        self.transform = transform
        self.aug_class_id = aug_class_id

    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, index): 
        x, y = super().__getitem__(index)
        
        if self.aug_class_id is not None and self.transform is not None:
            if y in self.aug_class_id:
                x = self.transform(x)
        
        return x, y

In [ ]:
# figure out the indices of classes that need augmentation
aug_class_id = []

for id in range(1, 154):
    n_img = nums.get(str(id))
    if n_img < 927: # 75% des classes (3eme quartile)
        aug_class_id.append(id)


In [ ]:
# data augmentation transform
class RandomFlipsTransform:

    def __init__(self, identity_proba=0.5) -> None:
        self.identity_proba = identity_proba

    def __call__(self, img):
        if torch.rand(1) < self.identity_proba:
            return img
        return self.change_image(img)

    def change_image(self, img):
        p = torch.rand(1)
        if p < 0.25:
            return TF.rotate(img, 90)
        elif p < 0.5:
            return TF.rotate(img, 180)
        elif p < 0.75:
            return TF.rotate(img, 270)
        return TF.gaussian_blur(img, 3)
        


aug_transforms = RandomFlipsTransform(0.66)

### Loading et splitting des données

In [ ]:
dataset = ImageFolder(train_dir_path, transforms.Compose([transforms.Resize([183, 183]), transforms.ToTensor()]))
# dataset = SmartAugmentationDataset(train_dir_path, transform=aug_transforms, aug_class_id=aug_class_id)

In [ ]:
train_proportion = 0.9
# train_len = int(len(dataset)*train_proportion)
# valid_len = len(dataset) - train_len

# train_split, valid_split = random_split(dataset, [train_len, valid_len])


# split les indices
train_indices, valid_indices, _, _ = train_test_split(
    range(len(dataset)),
    dataset.targets,
    stratify=dataset.targets,
    test_size=1 - train_proportion
)

# split le dataset avec les indices
train_split = Subset(dataset, train_indices)
valid_split = Subset(dataset, valid_indices)

In [ ]:
class_weights = {int(class_name):1/class_len for (class_name, class_len) in nums.items()}

In [ ]:
sample_weights = []
for i in tqdm(range(len(train_split))):
    _, y = train_split.__getitem__(i)
    sample_weights.append(class_weights[y+1])

In [ ]:
batch_size = 16 # valeur un peu au hasard pour le moment


In [ ]:
# num workers = 2 parce que le cpu de kaggle a 2 cores (2-core Intel(R) Xeon(R) CPU @ 2.30GHz)
# pin memory pour rendre les epoch plus rapide apres la premiere (bof)

# pas besoin de shuffle le validation set vu que l'eval est pas influencee par l'ordre des batches

train_sampler = WeightedRandomSampler(sample_weights, 2*len(sample_weights), replacement=True)
train_dl = DataLoader(train_split, batch_size, num_workers=2, sampler=train_sampler)
valid_dl = DataLoader(valid_split, batch_size, num_workers=2)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available else torch.device("cpu")

In [ ]:
# Pour ne pas avoir a choisir 
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

## Network

In [ ]:
# calcul de la justesse du modele
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


# general model methods
class PlantNetModel(nn.Module):
    
    def batch_training(self, batch):
        imgs, labels = batch
        out = self(imgs)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def batch_evaluation(self, batch):
        imgs, labels = batch
        out = self(imgs)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {"val_loss": loss.detach(), "val_accuracy": acc}
    
    def eval_epoch(self, outputs):
        batch_losses = [x["val_loss"] for x in outputs]
        batch_accuracy = [x["val_accuracy"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()  
        epoch_accuracy = torch.stack(batch_accuracy).mean()
        return {"val_loss": epoch_loss, "val_accuracy": epoch_accuracy}
    
    
    def print_epoch_end(self, epoch, result):
        print((f"""Epoch [{epoch+1}],
        train_loss: {result['train_loss']}, 
        val_loss: {result['val_loss']}, 
        val_acc: {result['val_accuracy']}""")
        )

In [ ]:
# NB: could try to add dropout layers down there

In [ ]:
# convolution block avec BatchNormalization
def ConvBlock(in_channels, out_channels, pool=False, kernel_size=3, padding=1, stride=1, pooling_kernel_size=3):
    
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)]
    if pool:
        layers.append(nn.MaxPool2d(pooling_kernel_size))
    return nn.Sequential(*layers)

# simpler net
class ConvNet(PlantNetModel):
    def __init__(self, in_channels, n_classes):
        super().__init__()
        
        self.conv1 = ConvBlock(in_channels, 64)
        self.conv2 = ConvBlock(64, 128, pool=True) # 128x61x61
        
        
        self.conv3 = ConvBlock(128, 256, pool=True) # 256 x 20 x 20
        self.conv4 = ConvBlock(256, 512, pool=True) # 512 x 6 x 6
        self.conv5 = ConvBlock(512, 512, pool=True) # 512 x 2 x 2
        
        self.dropout = nn.Dropout(p=0.1) 
        
        self.classif = nn.Sequential(nn.MaxPool2d(2),
                                     nn.Flatten(),
                                     nn.Linear(512, n_classes))
        
    def forward(self, batch):
        out = self.conv1(batch)
        out = self.conv2(out)
        out = self.dropout(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.dropout(out)
        out = self.conv5(out)
        out = self.classif(out)
        return out

# net architecture 
class ResNet(PlantNetModel):
    def __init__(self, in_channels, n_classes):
        super().__init__()
        
        self.conv1 = ConvBlock(in_channels, 32)
        self.conv2 = ConvBlock(32, 64, pool=True) # 64x61x61
        self.conv3 = ConvBlock(64, 128, pool=True) # 128 x 20 x 20
        self.res1 = nn.Sequential(ConvBlock(128, 128), ConvBlock(128, 128), ConvBlock(128, 128))

        self.conv4 = ConvBlock(128, 256, pool=True) # 256 x 6 x 6
        self.conv5 = ConvBlock(256, 256, pool=True, pooling_kernel_size=2) # 256 x 3 x 3
        self.res2 = nn.Sequential(ConvBlock(256, 256), ConvBlock(256, 256), ConvBlock(256, 256))
        
        self.classif = nn.Sequential(nn.MaxPool2d(3),
                                     nn.Flatten(),
                                     nn.Linear(256, n_classes))
        
        self.dropout = nn.Dropout(p=0.1)
        
    def forward(self, batch):
        out = self.conv1(batch)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.res1(out) + out
        out = self.dropout(out)
        out = self.conv4(out)
        out = self.dropout(out)
        out = self.conv5(out)
        out = self.dropout(out)
        out = self.res2(out) + out
        out = self.classif(out)
        return out

## Training

In [ ]:
model = ConvNet(3, len(dataset.classes))
# model = ResNet(3, len(dataset.classes))
model.cuda()

In [ ]:
torch.save(model.state_dict(), "best_model.pt") # save the untrained model on sait jamais

In [ ]:
# for training
@torch.no_grad()
def evaluate(model, val_dl, prog_bar=False):
    model.eval() # evaluation mode
    if prog_bar:
        outputs = [model.batch_evaluation(to_device(batch, device)) for batch in tqdm(val_dl)]
    else:
        outputs = [model.batch_evaluation(to_device(batch, device)) for batch in val_dl]
    return model.eval_epoch(outputs)
    
    
def full_training(epochs, 
                  lr, 
                  model, 
                  train_dl, 
                  val_loader, 
                  weight_decay=0, 
                  grad_clip=None, 
                  optimizer=torch.optim.SGD):
    
    torch.cuda.empty_cache()
    history = []
    best_val_acc = 0
    
    optimizer = optimizer(model.parameters(), lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, mode="max", verbose=True)
    # sched = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, verbose=True)
    
    for epoch in range(epochs):
        # Training
        # print(f"Started training epoch {epoch+1}...")
        model.train() # training mode
        train_losses = []
        for batch in tqdm(train_dl):
            batch = to_device(batch, device)
            
            loss = model.batch_training(batch)
            train_losses.append(loss)
            loss.backward()
            
            # gradient clipping (pour le resnet)
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
                
            optimizer.step()
            optimizer.zero_grad()

        # validation
        result = evaluate(model, val_loader, prog_bar=False)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.print_epoch_end(epoch, result)
        history.append(result)
        vc = result["val_accuracy"]
        sched.step(vc) # for reduce lr on plateau
        # sched.step()
        if  vc > best_val_acc:
            print(f"Saving new best model from epoch {epoch+1} with val_acc of {vc}\n")
            torch.save(model.state_dict(), "best_model.pt")
    
    return history

In [ ]:
epochs = 10
lr = 0.01
grad_clip = 5 # just in case there is a bad minibatch
# weight_decay = 1e-4
optim = torch.optim.Adam # voir si SGD ou Adamw marche mieux

In [ ]:
history = [evaluate(model, valid_dl, prog_bar=True)]

In [ ]:
history # bad accuracy because the weights are still random

In [ ]:
history += full_training(epochs, lr, model, train_dl, valid_dl, grad_clip=grad_clip, optimizer=optim)

## Experiments

## Progress

- CNN, 10 epochs, lr=0.1, 5 blocs cnn et 1 bloc de sortie fcn : val_acc= 0.68

In [ ]:
model = ResNet(3, len(dataset.classes))
model.load_state_dict(torch.load("best_model.pt"))
model.cuda()
model.eval()

In [ ]:
import pandas as pd

# we need to preserve order maybe ? update: we don't
orig_sub = pd.read_csv("../input/polytech-nice-data-science-course-2021/polytech/sample_submission.csv")

In [ ]:
def pred(img_name):
    img = Image.open("../input/polytech-nice-data-science-course-2021/polytech/test/" + img_name)
    in_tr = transforms.Compose([transforms.Resize([183,183]), transforms.ToTensor()])
    img = in_tr(img)
    
    img_batch = img.unsqueeze(0).cuda()
    out = model(img_batch)
    _, pred_class = torch.max(out, dim=1)
    return pred_class

In [ ]:
submission = pd.DataFrame(columns=["image_name", "class"])

for i, img_name in enumerate(tqdm(orig_sub["image_name"])):
    submission.at[i, "image_name"] = img_name
    submission.at[i, "class"] = dataset.classes[pred(img_name).item()]

In [ ]:
submission.head(10)

In [ ]:
submission["class"].value_counts()

In [ ]:
submission.to_csv("submission.csv", index=False)

In [ ]:
pred("1.jpg")

## Conclusion